<a href="https://colab.research.google.com/github/agnel-kf/Churn_Model/blob/main/Churn_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import tensorflow as tf
import numpy as np

## Data Preprocessing

In [35]:
dataset = pd.read_csv('/Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values
y.reshape(len(y),1)
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


# Encoding
## Label Encoding

In [36]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


## One Hot Encoding

In [37]:
ct = ColumnTransformer( transformers = [
                         ('encoder',OneHotEncoder(),[1])
],remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [38]:
from sklearn.model_selection import train_test_split
train_X,val_X,train_y,val_y = train_test_split(X,y,test_size = 0.25,random_state = 1)

# Feature Scaling

In [39]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_X = sc.fit_transform(train_X)
val_X = sc.transform(val_X)

In [40]:
print(train_X)

[[ 1.00080032 -0.58207179 -0.57324462 ...  0.64262086  0.98333878
  -1.48050131]
 [ 1.00080032 -0.58207179 -0.57324462 ...  0.64262086 -1.01694352
  -1.56523002]
 [-0.99920032 -0.58207179  1.74445599 ...  0.64262086 -1.01694352
  -1.18210827]
 ...
 [ 1.00080032 -0.58207179 -0.57324462 ...  0.64262086  0.98333878
  -0.13896318]
 [ 1.00080032 -0.58207179 -0.57324462 ...  0.64262086  0.98333878
   0.01979185]
 [-0.99920032  1.71800114 -0.57324462 ...  0.64262086 -1.01694352
  -1.15605488]]


# Building the model
## Initialising the model

In [41]:
ann = tf.keras.models.Sequential()

## Adding the input layer and hidden layer

In [42]:
ann.add(tf.keras.layers.Dense(units = 6,activation = 'relu'))

## Adding the second hidden layer

In [43]:
ann.add(tf.keras.layers.Dense(units = 6,activation = 'relu'))

## Adding the output layer

In [44]:
ann.add(tf.keras.layers.Dense(units = 1,activation = 'sigmoid'))

# Training ANN on the Training Test
## Compiling the ANN

In [45]:
ann.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [46]:
ann.fit(train_X,train_y,batch_size =32,epochs = 100 )

Epoch 1/100
235/235 [==============================] - 1s 3ms/step - loss: 0.5551 - accuracy: 0.7421
Epoch 2/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4571 - accuracy: 0.8025
Epoch 3/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4399 - accuracy: 0.8109
Epoch 4/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4300 - accuracy: 0.8140
Epoch 5/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4234 - accuracy: 0.8160
Epoch 6/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4182 - accuracy: 0.8184
Epoch 7/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4133 - accuracy: 0.8227
Epoch 8/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4079 - accuracy: 0.8253
Epoch 9/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4028 - accuracy: 0.8293
Epoch 10/100
235/235 [==============================] - 1s 3ms/step - loss: 0.3976 - accura

# Predicting and Evaluating the Test Set

In [47]:
y_preds = ann.predict(val_X)
y_preds = (y_preds > 0.5)
print(np.concatenate((y_preds.reshape(len(y_preds),1), val_y.reshape(len(val_y),1)),1))

[[0 0]
 [0 0]
 [0 0]
 ...
 [1 1]
 [0 0]
 [0 0]]


## Predicting an individual row

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [49]:

example_prediction = ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))
print(example_prediction> 0.5)

[[False]]


## Evaluating the model

In [50]:
from sklearn.metrics import confusion_matrix,accuracy_score
ct = confusion_matrix(val_y,y_preds)
print(ct)
accuracy_score(val_y,y_preds)

[[1909   71]
 [ 268  252]]


0.8644